In [ ]:
import pandas as pd
import numpy as np
import numpy as np
import re
import seaborn as sns
from scipy import stats
import math
import operator
import plotly
import plotly.plotly as py
import plotly.tools as tls
import plotly.graph_objs as go
from plotly.graph_objs import *
from plotly.offline import init_notebook_mode, iplot, plot

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt
plt.rcParams['figure.figsize'] = 10, 10

from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV, Ridge, RidgeCV
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
import feather
df = feather.read_dataframe("E:/Users/Johana/Documents/DH/Trabajo Final/principal.feather")
df.sample(2)

In [ ]:
columnas_principales = ['price_aprox_usd','property_type','surface_covered_in_m2','provincia']
df_reg = df[columnas_principales]
df_modelo= df_reg.dropna(axis =0 , how = 'any', subset = columnas_principales)
df_modelo.sample(2)

In [ ]:
#reviso distribución de variable dependiente

sns.distplot(df_modelo['price_aprox_usd'])

In [ ]:
df_dummies = pd.get_dummies(df_modelo, columns = [ 'provincia', 'property_type'], drop_first = True)
df_dummies.columns

In [ ]:
y = pd.DataFrame(df_dummies.price_aprox_usd)

x= df_dummies.drop(columns=['price_aprox_usd'])

In [ ]:
#reviso la varianza para ver si es necesario normalizar
print([x[col].var() for col in x.columns])

In [ ]:
#normalizacion por std
from sklearn import datasets, preprocessing
stdscaler = preprocessing.StandardScaler()
xs2 = stdscaler.fit_transform(x)

In [ ]:
#Convierto en Dataframe
X = pd.DataFrame(xs2, columns=x.columns)
X.sample(2)

In [ ]:
# reviso varianza despues de normalizar y veo que todas las variables quedaron con varianza 1
print([X[col].var() for col in X.columns])

In [ ]:
#estudiamos VIF para cuantificar la intensidad de la multicolinealidad
#Si VIF>10, la multicolinealidad es alta

import statsmodels.stats.outliers_influence as oi

for i in range(len(X.columns)):
    vif_col = oi.variance_inflation_factor(np.matrix(X), i)
    print('columna ' + str(i) + " " + str(vif_col))

In [ ]:
import statsmodels.api as sm
#agrego el intercepto
X_intercept = sm.add_constant(X)

y.index = range(y.shape[0])

# Fit and summarize OLS model (OLS = ordinary least square linear regression) Veo las descritivas del modelo
model_intercept = sm.OLS(y, X_intercept)
model_intercept = model_intercept.fit()
print (model_intercept.summary())
predictions_intercept = model_intercept.predict(X_intercept)

In [ ]:
#Calculo de residuos

predictions_intercept_df = pd.DataFrame(predictions_intercept, columns=['price_aprox_usd'])
residuos_intercept = y - predictions_intercept_df
print(residuos_intercept.mean())

In [ ]:
#verifico la distribución y que la media de los residuos sea cero
sns.distplot(residuos_intercept)

In [ ]:
#verifico homocedasticidad con base en p_value
#supuesto: Para cualquier valor de la variable explicativa, el error tienen la misma varianza
import statsmodels.stats.api as sms

resids_standardized = model_intercept.get_influence().resid_studentized_internal

resids = model_intercept.resid

bp_test = pd.DataFrame(sms.het_breuschpagan(resids, model_intercept.model.exog), 
                       columns=['value'],
                       index=['Lagrange multiplier statistic', 'p-value', 'f-value', 'f p-value'])


gq_test = pd.DataFrame(sms.het_goldfeldquandt(resids, model_intercept.model.exog)[:-1],
                       columns=['value'],
                       index=['F statistic', 'p-value'])

print('\n Breusch-Pagan test ----')
print(bp_test)
print('\n Goldfeld-Quandt test ----')
print(gq_test)
#Si todo es consistente los dos test deberian dar el mismo resultado (significativo o no)

In [ ]:
#Vemos los gráficos de los residuos, no obtengo una linea horizontal (supuesto de homocedasticidad)
fig, ax = plt.subplots(1,2)

fitted_vals = predictions_intercept

sns.regplot(x=fitted_vals, y=resids, lowess=True, ax=ax[0], line_kws={'color': 'red'})
ax[0].set_title('Residuals vs Fitted', fontsize=16)
ax[0].set(xlabel='Fitted Values', ylabel='Residuals')

sns.regplot(x=fitted_vals, y=np.sqrt(np.abs(resids_standardized)), lowess=True, ax=ax[1], line_kws={'color': 'red'})
ax[1].set_title('Scale-Location', fontsize=16)
ax[1].set(xlabel='Fitted Values', ylabel='sqrt(abs(Residuals))')

In [ ]:
# verifico si los residuos tienen distribucion normal. (con un grafico de datos reales vs simulados). Deberia dar una recta.
#el grafico es en quantiles

sm.ProbPlot(model_intercept.resid).qqplot(line='s');
plt.title('Q-Q plot');

jb = stats.jarque_bera(model_intercept.resid)
sw = stats.shapiro(model_intercept.resid)
ad = stats.anderson(model_intercept.resid, dist='norm')
ks = stats.kstest(model_intercept.resid, 'norm')

print(f'Jarque-Bera test ---- statistic: {jb[0]:.4f}, p-value: {jb[1]}')
print(f'Shapiro-Wilk test ---- statistic: {sw[0]:.4f}, p-value: {sw[1]:.4f}')
print(f'Kolmogorov-Smirnov test ---- statistic: {ks.statistic:.4f}, p-value: {ks.pvalue:.4f}')
print(f'Anderson-Darling test ---- statistic: {ad.statistic:.4f}, 5% critical value: {ad.critical_values[2]:.4f}')
print('If the returned AD statistic is larger than the critical value, then for the 5% significance level, the null hypothesis that the data come from the Normal distribution should be rejected.')

In [ ]:
#verifico el R2 y el error de la regresion
print('r2: ' + str(model_intercept.rsquared))

In [ ]:
#Divido los datos en train and test

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=53)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
# Calculamos el RMSE

lm = LinearRegression(fit_intercept=True)
lm.fit(X_train, y_train)

y = y_train
y_pred = lm.predict(X_train)



#definimos la raiz del error cuadratico medio
rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))
print(" Score Test Lineal: %.2f\n" % lm.score(X_train, y_train))
print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred))

In [ ]:
#los alfas se crean para hacer un tunning de los hiperparámetros de los modelos en base a validación cruzada
#despues creo la regresion y le paso el alfa que obtuve

kf = KFold(n_splits=5, shuffle=True)
alfas_ridge = np.linspace(0.001, 0.3, 300)
lm_ridge_cv= RidgeCV(alphas=alfas_ridge, cv=kf, normalize=False, fit_intercept = True)
lm_ridge_cv.fit(X_train, y_train)
print('Alpha Ridge:', lm_ridge_cv.alpha_)   

#utilizo el alfa del paso anterior y  fiteo con un intercepto

model_ridge = Ridge(lm_ridge_cv.alpha_, normalize=False, fit_intercept = True)
model_ridge.fit(X_train, y_train)


y_pred_ridge = model_ridge.predict(X_train)
np.sqrt(mean_squared_error(y_train, y_pred_ridge))

In [ ]:
#pruebo lo mismo que en el paso anterior pero con lasso para verificar consistencia y encontrar el mejor modelo
alfas_lasso = np.linspace(0.1, 0.5, 300)
lm_lasso_cv = LassoCV(alphas=alfas_lasso, cv=kf, normalize=False, fit_intercept = True)
lm_lasso_cv.fit(X_train, y_train)
print('Alpha LASSO:', lm_lasso_cv.alpha_)


model_lasso = Lasso(lm_lasso_cv.alpha_, normalize=False, fit_intercept = True)
model_lasso.fit(X_train, y_train)
y_pred_lasso = model_lasso.predict(X_train)
np.sqrt(mean_squared_error(y_train, y_pred_lasso))

In [ ]:
# Calculamos el R2 para ridge y Lasso

print("Score Train Ridge : %.2f\n" % lm_ridge_cv.score(X_train, y_train),
      "Score Train Lasso : %.2f\n" %  lm_lasso_cv.score(X_train, y_train))

# Calculamos el RMSE
rmse = lambda y, y_pred: np.sqrt(mean_squared_error(y, y_pred))


print(" Train RMSE lineal   : %.2f \n" % rmse(y_train,y_pred_lm),
      "Train RMSE Ridge    : %.2f \n" % rmse(y_train,y_pred_ridge),
      "Train RMSE Lasso    : %.2f \n" % rmse(y_train,y_pred_lasso))

In [ ]:
#Hacemos el calculo con los datos de test (score y error cuadratico medio)

lm.fit(X_test, y_test)

y_pred_lmtest = lm.predict(X_test)
print(" Score Test Lineal: %.2f\n" % lm.score(X_test, y_test))
print(" Test RMSE lineal= %.2f\n" % rmse(y_test, y_pred_lmtest))

In [ ]:
#Pruebo con test ridge
lm_ridge_cv.fit(X_test, y_test)
print('Alpha Ridge:', lm_ridge_cv.alpha_) 

y_pred_ridgetest = model_ridge.predict(X_test)

np.sqrt(mean_squared_error(y_test, y_pred_ridgetest))

In [ ]:
#Pruebo con test lasso
model_lasso.fit(X_test, y_test)
y_pred_lassotest = model_lasso.predict(X_test)
np.sqrt(mean_squared_error(y_test, y_pred_lassotest))

In [ ]:
# valores para test


print("Score Test Ridge : %.2f\n" % lm_ridge_cv.score(X_test, y_test),
      "Score Test Lasso : %.2f\n" %  lm_lasso_cv.score(X_test, y_test))


print("Test RMSE Ridge = %.2f\n" %  rmse(y_test, y_pred_ridgetest),
      "Test RMSE Lasso = %.2f" %  rmse(y_test, y_pred_lassotest))

In [ ]:
#deben compararse el score, el r2 y el rsme de los tres modelos para sacar alguna conclusión y ponerla en la ppt